In [1]:
# basic libraries needed
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import copy
import tensorflow as tf
import keras.api._v2.keras as keras
from keras import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import array

In [2]:
data_hor_disc = pd.read_excel("Data/DiscreteMotion_Data_Horizontalsetup.xlsx", sheet_name = "Sheet1")
data_hor_cont = pd.read_excel("Data/ContinuousMotion_Data_Horizontalsetup.xlsx", sheet_name = "Sheet1")
# data_ver_disc = pd.read_excel("Data/DiscreteMotion_Data_Verticalsetup.xlsx", sheet_name = "Tabelle1")
# data_ver_cont = pd.read_excel("Data/ContinuousMotion_Data_Verticalsetup.xlsx", sheet_name = "Tabelle1")

data_hor_disc=data_hor_disc.add_suffix('_disc')
data_hor_cont=data_hor_cont.add_suffix('_cont')

In [3]:
data_hor_disc.head()

,Pow_100_disc,Pow_200_disc,Pow_300_disc,Pow_400_disc,Pow_500_disc,Pow_600_disc,Pow_700_disc,Pow_800_disc,Pow_900_disc,Pow_1000_disc,Pow_1100_disc,Pow_1200_disc,Pow_1300_disc,Pow_1400_disc,Pow_1500_disc,Pow_1600_disc,Pow_1700_disc,Pow_1800_disc,Pow_1900_disc,Pow_2000_disc
0,0.846,0.044,0.014,0.255,4.009,0.516,5.616,7.860,9.240,9.974999,10.981999,6.578000,12.059999,7.950,12.599,9.990001,9.555000,19.096001,13.938000,0.202
1,0.846,0.044,0.167,0.255,4.009,0.516,5.616,7.860,9.240,9.974999,10.981999,6.578000,12.059999,7.950,12.599,9.990001,9.555000,19.096001,13.938000,0.202
2,0.846,0.044,0.167,0.255,4.009,5.676,5.400,6.930,9.240,9.974999,9.728000,6.578000,11.834999,7.830,12.599,9.990001,10.465000,13.454000,11.039999,0.564
3,0.774,0.147,0.167,4.080,4.199,5.676,5.400,6.930,7.854,8.014999,9.728000,6.578000,11.834999,7.830,7.267,9.287000,10.465000,13.454000,11.039999,0.564
4,0.774,0.147,2.171,4.080,4.199,5.676,5.400,6.468,7.854,8.014999,9.472000,10.372999,11.834999,12.267,7.267,9.287000,15.548001,13.454000,11.280000,10.434


In [4]:
data_hor_cont.head()

,Pow_100_cont,Pow_200_cont,Pow_300_cont,Pow_400_cont,Pow_500_cont,Pow_600_cont,Pow_700_cont,Pow_800_cont,Pow_900_cont,Pow_1000_cont,Pow_1100_cont,Pow_1200_cont,Pow_1300_cont,Pow_1400_cont,Pow_1500_cont,Pow_1600_cont,Pow_1700_cont,Pow_1800_cont,Pow_1900_cont,Pow_2000_cont
0,0.210,1.557,1.86,3.859,5.300,2.970,3.424,6.625,4.392,8.365000,4.320000,6.072000,8.744999,7.905,15.911999,12.314,15.749999,18.054001,20.286001,15.548001
1,0.210,1.557,1.86,3.859,5.300,2.970,3.424,6.625,4.392,8.365000,4.320000,6.072000,8.744999,7.905,15.911999,12.314,15.749999,18.054001,20.286001,15.548001
2,0.286,1.341,1.86,3.859,4.020,3.164,3.328,6.625,4.158,8.014999,10.943999,5.808000,8.744999,7.905,12.954000,13.886,13.600000,18.054001,20.286001,15.548001
3,0.286,1.341,1.73,3.230,4.020,3.165,3.328,5.300,4.158,8.014999,10.943999,5.808000,8.217000,7.874,12.954000,13.886,13.600000,17.582001,19.467001,16.483999
4,0.858,1.490,1.73,3.230,3.819,4.642,5.408,5.300,7.392,7.786000,10.943999,9.921999,8.217000,7.874,12.954000,13.886,15.232000,17.582001,19.467001,16.483999


In [5]:
# result = pd.DataFrame(list(zip(data_hor_disc.columns, data_hor_cont.columns)))
# # uniamo le righe
# # result = pd.concat([data_hor_disc, data_hor_cont], axis=1)

# result = pd.concat([data_hor_disc, data_hor_cont], axis=1, ignore_index=False)

result = pd.DataFrame()

for i in range(len(data_hor_disc.columns)):
    result[data_hor_disc.columns[i]] = data_hor_disc[data_hor_disc.columns[i]]
    result[data_hor_cont.columns[i]] = data_hor_cont[data_hor_cont.columns[i]]

In [6]:
result.head()

,Pow_100_disc,Pow_100_cont,Pow_200_disc,Pow_200_cont,Pow_300_disc,Pow_300_cont,Pow_400_disc,Pow_400_cont,Pow_500_disc,Pow_500_cont,...,Pow_1600_disc,Pow_1600_cont,Pow_1700_disc,Pow_1700_cont,Pow_1800_disc,Pow_1800_cont,Pow_1900_disc,Pow_1900_cont,Pow_2000_disc,Pow_2000_cont
0,0.846,0.210,0.044,1.557,0.014,1.86,0.255,3.859,4.009,5.300,...,9.990001,12.314,9.555000,15.749999,19.096001,18.054001,13.938000,20.286001,0.202,15.548001
1,0.846,0.210,0.044,1.557,0.167,1.86,0.255,3.859,4.009,5.300,...,9.990001,12.314,9.555000,15.749999,19.096001,18.054001,13.938000,20.286001,0.202,15.548001
2,0.846,0.286,0.044,1.341,0.167,1.86,0.255,3.859,4.009,4.020,...,9.990001,13.886,10.465000,13.600000,13.454000,18.054001,11.039999,20.286001,0.564,15.548001
3,0.774,0.286,0.147,1.341,0.167,1.73,4.080,3.230,4.199,4.020,...,9.287000,13.886,10.465000,13.600000,13.454000,17.582001,11.039999,19.467001,0.564,16.483999
4,0.774,0.858,0.147,1.490,2.171,1.73,4.080,3.230,4.199,3.819,...,9.287000,13.886,15.548001,15.232000,13.454000,17.582001,11.280000,19.467001,10.434,16.483999


In [8]:
# result = result.drop('Unnamed: 0', axis=1)
result.to_csv("Data/combined_data_hor.csv")

In [4]:
data_ver_disc = pd.read_excel("Data/DiscreteMotion_Data_Verticalsetup.xlsx", sheet_name = "Tabelle1")
data_ver_cont = pd.read_excel("Data/ContinuousMotion_Data_Verticalsetup.xlsx", sheet_name = "Tabelle1")
# data_ver_disc = pd.read_excel("Data/DiscreteMotion_Data_Verticalsetup.xlsx", sheet_name = "Tabelle1")
# data_ver_cont = pd.read_excel("Data/ContinuousMotion_Data_Verticalsetup.xlsx", sheet_name = "Tabelle1")

data_ver_disc=data_ver_disc.add_suffix('_disc')
data_ver_cont=data_ver_cont.add_suffix('_cont')

In [5]:

result_ver = pd.DataFrame()

for i in range(len(data_ver_disc.columns)):
    result_ver[data_ver_disc.columns[i]] = data_ver_disc[data_ver_disc.columns[i]]
    result_ver[data_ver_cont.columns[i]] = data_ver_cont[data_ver_cont.columns[i]]

In [6]:
result_ver.to_csv("Data/combined_data_ver.csv")